In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install -q transformers==4.4.2 sentencepiece

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [10]:
MODEL_DIR = '/content/drive/MyDrive/Colab Notebooks/akimoto/nlp'

# Models (SentencePiece)
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)


In [11]:
import argparse

# 各種ハイパーパラメータ
args_dict = dict(
    max_input_length=64,
    max_target_length=512,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=10,
)

args = argparse.Namespace(**args_dict)


In [13]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s


In [14]:
def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

In [19]:
import textwrap

MAX_SOURCE_LENGTH = args.max_input_length   # 入力される記事本文の最大トークン数
MAX_TARGET_LENGTH = args.max_target_length  # 生成されるタイトルの最大トークン数

def preprocess_title(text):
    return normalize_text(text.replace("\n", ""))

# 推論モード設定
trained_model.eval()

def generate_texts(title):
    # 前処理とトークナイズを行う
    inputs = [preprocess_title(title)]
    batch = tokenizer.batch_encode_plus(
        inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
        padding="longest", return_tensors="pt")

    input_ids = batch['input_ids']
    input_mask = batch['attention_mask']

    USE_GPU = False
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()
    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=input_ids, attention_mask=input_mask, 
        max_length=MAX_TARGET_LENGTH,
        # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        # num_beams=10,  # ビームサーチの探索幅
        # diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        # num_beam_groups=10,  # ビームサーチのグループ
        # num_return_sequences=10,  # 生成する文の数
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
    )

    # 生成されたトークン列を文字列に変換する
    generated_bodies = [tokenizer.decode(ids, skip_special_tokens=True, 
                                        clean_up_tokenization_spaces=False) 
                        for ids in outputs]

    # 生成された文章を表示する
    for i, body in enumerate(generated_bodies):
        print("\n".join(textwrap.wrap(f"{i+1:2}. {body}")))


In [21]:
generate_texts('I SEE...')

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1764: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


 1. eah i'm sorry...i'm sorry...愛はどこへ消えたのだろう?何度泣いても涙が止まらなくて悲しくて泣きたくなるよ
君に会いたいって思えるまでずっとそばにいたかっただけど、どうすればいいのか分からなかった。なぜだかわからないけど誰かと会ったことなんてないん
だから僕たちは何も言わないでしょ?誰にも言わず黙って見守ってくれるはずなのに誰も気づいてくれない恋なんかするわけないじゃん!


In [22]:
generate_texts('根も葉もRUMOR')

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1764: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


 1. ey!花は咲いてるのに何だか寂しくなるよ君と出会ってからもう1年経つんだね僕たちはいつの間にか仲良くなっちゃった恋なんかしたくなくて
さっきまで声を掛けてたけど今さら聞けるわけないじゃん根も葉もrumorみんなでワイワイ楽しく過ごした日々が嘘のように過ぎ去って行くその瞬間(
とき)をどう生きるのか?生き方なんて人それぞれだけど愛とは何か?それとも誰かと一緒にいたいどんな風に吹かれながら歩き出してみようかなあそびた
りして...
